In [29]:
import os
import music21 as m21
import json
import tensorflow as tf
import keras
import numpy as np
from ipynb.fs.full.Preprocessing import training_sequence, LENGTH, DICT_PATH

In [30]:
class MelodyGenerator:

    def __init__(self, model_path="misc/Network_model.h5"):

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open("misc/dict.json", "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * LENGTH


    def generateMelody(self, seed, num_steps, max_sequence_length, temperature):

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # (1, max_sequence_length, num of symbols in the vocabulary)
            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self._sample_with_temperature(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def _sample_with_temperature(self, probabilites, temperature):

        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index
    
    def saveMelody(self,melody, step_duration=0.25, format="midi", file_name="misc/melody/melody.mid"):
        # step_duration is the amount of duration in quarter note length that we have at each step in out time series representation.

        # create a stream
        stream = m21.stream.Stream()

        # Pasring through all the symbols in melody and creating note/rest objects

        start_symbol = None
        counter = 1
    
        for i, symbol in enumerate(melody):

            if symbol != "_" or i + 1 == len(melody):
            # Make sure that the first event is not being dealt
                if start_symbol is not None:

                    quarter_length_duration = step_duration * counter

                    if start_symbol == "r":
                        event = m21.note.Rest(quarterLength=quarter_length_duration)
                    else:
                        event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
                
                    stream.append(event)
                    # rest counter
                    counter = 1

                start_symbol = symbol

            else:
                counter += 1
                
    
        stream.write(format, file_name)

In [33]:
seed = "64 _ 64 _ 64 _ 69 _ "
g = MelodyGenerator()
melody = g.generateMelody(seed, 500, LENGTH, 0.8)
print(melody)
g.saveMelody(melody)

1/1 [==============================] - 0s 16ms/step


2023-02-14 18:43:42.907827: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-14 18:43:42.955639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 15ms/step
